In [11]:
import pickle
import numpy as np

participantCount = 32

rawData = [{}]*participantCount

for i in range(participantCount):
    rawData[i] = pickle.load(open(f'../../../data/s{i+1:02}.dat', 'rb'), encoding="latin1")

labels = np.array(list(map(lambda participant : participant['labels'], rawData)))
data = np.array(list(map(lambda participant : participant['data'], rawData)))

def get_y(emotion):
    return (labels.reshape(-1, 4)[:,emotion] >= 5).astype(int)

def get_peripheral_x():
    return data[:, :, 32:, :].reshape(-1, 8, 8064)

In [13]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import KFold
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Y = get_y(0)
X = np.moveaxis(get_peripheral_x(), 1, 2)

for i in range(4):
    print("EMOTION NUMBER", i)
    Y = get_y(i)
    kfold = KFold(n_splits=5, shuffle=False)
    cvscores = []
    for train, test in kfold.split(X, Y):
        model = Sequential()
        model.add(LSTM(100, input_dim=8))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc"])

        model.fit(X[train], Y[train], validation_data=(X[test], Y[test]), epochs=20, batch_size=120, verbose=2, shuffle=False)

        scores = model.evaluate(X[test], Y[test], verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)

    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))    
    print("EMOTION", i)

(40, 8, 8064)
[1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 1]
EMOTION NUMBER 0


C:\Users\wolukasz\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\wolukasz\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, input_shape=(None, 8))`



Train on 32 samples, validate on 8 samples
Epoch 1/20
 - 12s - loss: 0.7636 - acc: 0.5938 - val_loss: 0.9600 - val_acc: 0.2500
Epoch 2/20
 - 7s - loss: 0.7249 - acc: 0.6250 - val_loss: 0.9285 - val_acc: 0.2500
Epoch 3/20
 - 7s - loss: 0.7283 - acc: 0.6250 - val_loss: 0.9323 - val_acc: 0.2500
Epoch 4/20
 - 8s - loss: 0.7043 - acc: 0.6562 - val_loss: 0.9330 - val_acc: 0.2500
Epoch 5/20
 - 8s - loss: 0.6970 - acc: 0.6562 - val_loss: 0.9759 - val_acc: 0.1250
Epoch 6/20
 - 8s - loss: 0.6922 - acc: 0.6562 - val_loss: 1.0145 - val_acc: 0.1250
Epoch 7/20
 - 9s - loss: 0.6690 - acc: 0.6562 - val_loss: 1.0041 - val_acc: 0.1250
Epoch 8/20
 - 9s - loss: 0.6555 - acc: 0.6562 - val_loss: 1.0126 - val_acc: 0.1250
Epoch 9/20
 - 8s - loss: 0.6492 - acc: 0.6562 - val_loss: 1.0147 - val_acc: 0.1250
Epoch 10/20
 - 8s - loss: 0.6341 - acc: 0.6250 - val_loss: 1.0063 - val_acc: 0.1250
Epoch 11/20
 - 8s - loss: 0.6245 - acc: 0.6250 - val_loss: 0.9980 - val_acc: 0.1250
Epoch 12/20
 - 8s - loss: 0.6169 - acc: 

KeyboardInterrupt: 